In [8]:
import numpy as np
import jax.numpy as jnp
import scipy.stats as ss

# import matplotlib.pyplot as plt
# %matplotlib inline
# from IPython.display import display
# import sympy; sympy.init_printing()

# def display_matrix(m):
#     display(sympy.Matrix(m))

/Users/carolinepei/Desktop/mfe/project/citi/term_2_citi_project/citi/lib/python3.8/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


AttributeError: partially initialized module 'jax' has no attribute '_src' (most likely due to a circular import)

In [ ]:
N = 4          # number of time steps
r = 0.06       # interest rate
K = 1.1        # strike 
T = 3          # Maturity

dt = T/(N-1)          # time interval
df = jnp.exp(-r * dt)  # discount factor per time interval

In [ ]:
S = jnp.array([
            [1.00, 1.09, 1.08, 1.34],
            [1.00, 1.16, 1.26, 1.54],
            [1.00, 1.22, 1.07, 1.03],
            [1.00, 0.93, 0.97, 0.92],
            [1.00, 1.11, 1.56, 1.52],
            [1.00, 0.76, 0.77, 0.90],
            [1.00, 0.92, 0.84, 1.01],
            [1.00, 0.88, 1.22, 1.34]])

In [ ]:
H = jnp.maximum(K - S, 0)           # intrinsic values for put option
V = jnp.zeros_like(H)               # value matrix
V[:,-1] = H[:,-1]

# Valuation by LS Method
for t in range(N-2, 0, -1):

    good_paths = H[:,t] > 0        # paths where the intrinsic value is positive 
                                   # the regression is performed only on these paths 
    
    rg = jnp.polyfit( S[good_paths, t], V[good_paths, t+1] * df, 2)    # polynomial regression
    C = jnp.polyval( rg, S[good_paths,t] )                             # evaluation of regression  
    
    exercise = jnp.zeros( len(good_paths), dtype=bool)    # initialize
    exercise[good_paths] = H[good_paths,t] > C           # paths where it is optimal to exercise
    
    V[exercise,t] = H[exercise,t]                        # set V equal to H where it is optimal to exercise 
    V[exercise,t+1:] = 0                                 # set future cash flows, for that path, equal to zero  
    discount_path = (V[:,t] == 0)                        # paths where we didn't exercise 
    V[discount_path,t] = V[discount_path,t+1] * df       # set V[t] in continuation region
    
V0 = jnp.mean(V[:,1]) * df  # discounted expectation of V[t=1]
print("Example price= ", V0)

In [ ]:
print(H.round(2))
